# SEIR Optimization

Basado en pygmo:
https://esa.github.io/pygmo2/

In [ ]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pygmo as pg
import pandas as pd
from time import time

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR import SEIR
from importdata import ImportData as importdata
from Quarantine import Quarantine

## Create Pygmo Optimization Object

In [ ]:
class SEIROptimization:
    def __init__(self,Ir,tr,I_ac_r,I_ac_tr,tsim,alpha,population,bounds):
        self.Ir = Ir
        self.tr = tr
        self.I_ac_r = I_ac_r
        self.I_ac_tr = I_ac_tr
        self.tsim = tsim        
        self.alpha = alpha
        self.population = population        
        self.bounds = bounds
    def fitness(self,x):        
        sol = SEIR(tsim=self.tsim,alpha=self.alpha,beta=x[0],mu=x[1],k=x[2],I=self.Ir[0],I_ac=self.I_ac_r[0],population=self.population)
        sol.integr_sci(0,tsim,0.1)                
        idx=np.searchsorted(sol.t,self.tr)
        res = LA.norm(self.Ir-sol.I[idx])
        return([res])

    def get_bounds(self):
        return(self.bounds)

    def set_bounds(self,bounds):
        self.bounds = bounds
        return(self.bounds)


## Import Data

In [ ]:
# Import data: 
tstate = '13'
initdate = datetime(2020,5,15)

Ir,tr,Ir_dates = importdata.importActiveInfected(tstate = tstate, initdate = initdate)
I_ac_r,I_ac_tr,I_ac_dates = importdata.importAcumulatedInfected(tstate = tstate, initdate = initdate)
population = importdata.importPopulation(tstate = tstate)


## Optimization Params

In [ ]:
tsim = 1000
mob = 0.6
iqt = 0
alpha = Quarantine(mob,iqt=iqt).alpha

# Params to find
# beta,mu,k
lb=[0.01,0.1, 0]
ub=[   1,  2,30]
bounds = [lb,ub]

## Run Optimization

In [ ]:
opti = SEIROptimization(Ir=Ir,tr=tr,I_ac_r=I_ac_r,I_ac_tr=I_ac_tr,tsim=tsim,alpha=alpha,population=population,bounds=bounds)
algo = pg.algorithm(pg.pso(gen = 20))
pop = pg.population(opti,50)

In [ ]:
t0 = time()
pop = algo.evolve(pop)
t1 = time()
print('Optimization takes %f seconds' %(t1-t0))
print('Fitness Value')
print(pop.champion_f)
print('Optimal arguments')
print(pop.champion_x)

In [ ]:
pop.champion_x

# Single simulation example

In [ ]:
tsim = 1000
beta = 0.4
k = 1
mob = 0.6
alpha = Quarantine(mob).alpha
population = 1000000

In [ ]:
simulation = SEIR(tsim=tsim,alpha=alpha,beta=beta,mu=1,k=k,I=Ir[0],I_ac=I_ac_r[0],population=population)

In [ ]:
simulation.integr_sci(0,tsim,0.1)

## Plot =)

In [ ]:
endDay = 100
plt.plot(simulation.t[:endDay],simulation.S[:endDay],label='S')
plt.plot(simulation.t[:endDay],simulation.E[:endDay],label='E')
plt.plot(simulation.t[:endDay],simulation.I[:endDay],label='I')
plt.plot(simulation.t[:endDay],simulation.R[:endDay],label='R')
plt.legend(loc=0)
plt.show()